##### Carga del dataset #####

In [1]:
import numpy as np
import pandas as pd
import os, cv2

directory_path = '../Dataset/'
folders_list = []    
      
for item in os.listdir(directory_path):
    item_path = os.path.join(directory_path, item)
    if os.path.isfile(item_path):
        folders_list += [item]
    elif os.path.isdir(item_path):
        folders_list += [f'{item}:{len(os.listdir(item_path))}']        

print(f'Carpetas: {folders_list}')

Folders: ['A:70', 'B:70', 'C:70', 'D:70', 'E:70', 'F:70', 'G:70', 'H:70', 'I:70', 'J:70', 'K:70', 'L:70', 'M:70', 'N:70', 'O:70', 'P:70', 'Q:70', 'R:70', 'S:70', 'T:65', 'U:70', 'V:70', 'W:70', 'X:70', 'Y:70', 'Z:70']


##### Detección y visualización de landmarks #####

In [2]:
import mediapipe as mp

# Función para dibujar puntos de referencia a mano
def draw_landmarks_on_image(rgb_image, detection_result):
    hand_landmarks_list = detection_result.multi_hand_landmarks 
    annotated_image = np.copy(rgb_image) 

    for hand_landmarks in hand_landmarks_list:
        mp.solutions.drawing_utils.draw_landmarks(
            annotated_image,  # Imagen para dibujar
            hand_landmarks,  # Puntos clave para dibujar
            mp.solutions.hands.HAND_CONNECTIONS  # Conexiones entre puntos clave
        )

    return annotated_image

# Inizializza MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detecta los puntos clave de las manos
    results = hands.process(rgb_image)

    # Dibuja landmarks
    if results.multi_hand_landmarks:
        annotated_image = draw_landmarks_on_image(frame, results)
    else:
        annotated_image = frame 

    cv2.imshow('Hand Landmarks', annotated_image)

    # Presionar 'q' para salir
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

##### Extracción y guardado features #####

In [3]:
import csv

dataset_path = '../Dataset/'

# Inicialización MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.7)

# Archivo CSV para guardar las features
csv_file = '../KNN/asl_features.csv'

with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    # Iteración en las carpetas del dataset
    for label in sorted(os.listdir(dataset_path)):
        folder_path = os.path.join(dataset_path, label)
        if os.path.isdir(folder_path):
            for img_file in os.listdir(folder_path):
                img_path = os.path.join(folder_path, img_file)
                image = cv2.imread(img_path)

                rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                # Detección de landmarks
                results = hands.process(rgb_image)
                if results.multi_hand_landmarks:
                    # Extracción de características de landmarks
                    for hand_landmarks in results.multi_hand_landmarks:
                        features = []
                        for landmark in hand_landmarks.landmark:
                            features.extend([landmark.x, landmark.y, landmark.z])
                        # Agregando etiqueta de letra
                        features.append(label)
                        writer.writerow(features)


##### Subdivisión del dataset en Training y Test Set #####

In [4]:
from sklearn.model_selection import train_test_split

# Cargando el dataset
import pandas as pd
data = pd.read_csv('../KNN/asl_features.csv', header=None)
X = data.iloc[:, :-1].values  # Features
y = data.iloc[:, -1].values   # Etiquetas

# Subdivisión del dataset (80% training, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##### Buscar el valor óptimo de k #####

In [5]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

k_values = range(1, 21)
mean_accuracies = []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, y_train, cv=5)
    mean_accuracies.append(scores.mean())
    print(f'k={k}, Accuracy: {scores.mean():.4f}')

best_k = k_values[np.argmax(mean_accuracies)]
print(f'Mejor valor de k: {best_k}')

c:\Users\anixi\anaconda3\envs\VC_ENV\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


k=1, Accuracy: 0.9799
k=2, Accuracy: 0.9569


c:\Users\anixi\anaconda3\envs\VC_ENV\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\anixi\anaconda3\envs\VC_ENV\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


k=3, Accuracy: 0.9455
k=4, Accuracy: 0.9368


c:\Users\anixi\anaconda3\envs\VC_ENV\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\anixi\anaconda3\envs\VC_ENV\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


...

c:\Users\anixi\anaconda3\envs\VC_ENV\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


k=18, Accuracy: 0.8722


c:\Users\anixi\anaconda3\envs\VC_ENV\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


k=19, Accuracy: 0.8736
k=20, Accuracy: 0.8708
Mejor valor de k: 1


c:\Users\anixi\anaconda3\envs\VC_ENV\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


##### Evaluación del modelo con el valor k encontrado #####

In [6]:
# Inicialización del modelo
knn = KNeighborsClassifier(n_neighbors=best_k)
knn.fit(X_train, y_train)

# Evaluación en el test set
accuracy = knn.score(X_test, y_test)
print(f'Precisión en el test set: {accuracy:.4f}')

Precisión en el test set: 0.9657


##### Reconocimiento después de hacer clic en el espacio #####

In [7]:
import pickle

# cargando el modelo KNN entrenado
with open('../Models/knn_model_asl.pkl', 'rb') as f:
    knn = pickle.load(f)

# Inicialización MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Función para normalizar puntos de referencia
def normalize_features(hand_landmarks):
    x_coords = [landmark.x for landmark in hand_landmarks.landmark]
    y_coords = [landmark.y for landmark in hand_landmarks.landmark]

    # Cálculo del bounding box de la mano
    min_x, max_x = min(x_coords), max(x_coords)
    min_y, max_y = min(y_coords), max(y_coords)

    # Normalización con respecto al bounding box
    normalized_features = []
    for landmark in hand_landmarks.landmark:
        norm_x = (landmark.x - min_x) / (max_x - min_x)
        norm_y = (landmark.y - min_y) / (max_y - min_y)
        normalized_features.extend([norm_x, norm_y, landmark.z])  # Puoi includere o escludere Z a seconda dei test
    return normalized_features

# Variable para habilitar/deshabilitar la predicción
recognition_active = False

cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_image)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Dibuja los landmarks
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            if recognition_active:
                # Extracción de feature y predicción 
                features = []
                for landmark in hand_landmarks.landmark:
                    features.extend([landmark.x, landmark.y, landmark.z])
                prediction = knn.predict([features])
                
                cv2.putText(frame, f'Previsto: {prediction[0]}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    cv2.imshow('Reconocimiento de lengua de signos', frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):  # Salir del programa
        break
    elif key == ord(' '):  # Activar/desactivar el reconocimiento con la tecla 'space'
        recognition_active = not recognition_active
        print(f'Reconocimiento {"activado" if recognition_active else "desactivado"}')

cap.release()
cv2.destroyAllWindows()


Reconocimiento activado
